In [2]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import re
import numpy as np
import yaml
from argparse import ArgumentParser

from transformers import BertTokenizer, BertForMaskedLM
from datasets.quora_bert_mask_predict_dataset import QuoraBertMaskPredictDataset as Dataset
from utils import same_seeds

In [6]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
# parse argument
parser = ArgumentParser()
parser.add_argument("--config_path", dest="config_path",
                    default='../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml')
parser.add_argument("--seed", dest="seed", default=0, type=int)

args = parser.parse_args()
config_path = args.config_path
seed = args.seed
print("config_path:", config_path)
print("seed: ", seed)

same_seeds(seed)

NameError: name 'ArgumentParser' is not defined

In [3]:
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow.yaml'
config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk.yaml'
# config_path = '../configs/dpng_transformer_bert_tokenizer_bow_indivtopk_onlybow.yaml'


with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)
    
train_size = config['train_size']
val_size = config['val_size']
test_size = config['test_size']

batch_size = 128

is_bow = config['is_bow']
bow_strategy = config['bow_strategy']
topk = config['topk']
if bow_strategy != 'simple_sum':
    indiv_topk = config['indiv_topk']
else:
    # not used but use default value for simplicity
    indiv_topk = 50


try:
    is_bow = config['is_bow']

    if is_bow:
        bow_strategy = config['bow_strategy']
        topk = config['topk']
        if bow_strategy != 'simple_sum':
            indiv_topk = config['indiv_topk']
        else:
            # not used but use default value for simplicity
            indiv_topk = 50
        
        only_bow = config['only_bow']
        replace_predict = config['replace_predict']


except KeyError:
    is_bow = False


{'save_model_path': '../models/DNPG_base_transformer_bert_tokenizer_bert_bow_indivtopk.pth', 'log_file': '../logs/DNPG_base_transformer_bert_tokenizer_training_bert_bow_indiv_topk.txt', 'test_output_file': '../outputs/test_DNPG_transformer_bert_tokenizer_bow_indivtopk_out.txt', 'val_output_file': '../outputs/val_DNPG_transformer_bert_tokenizer_bow_indivtopk_out.txt', 'dataset': 'quora_bert_mask_predict_dataset', 'num_epochs': 50, 'batch_size': 100, 'd_model': 450, 'd_inner_hid': 512, 'd_k': 50, 'd_v': 50, 'n_head': 9, 'n_layers': 3, 'n_warmup_steps': 12000, 'dropout': 0.1, 'embs_share_weight': True, 'proj_share_weight': True, 'label_smoothing': False, 'train_size': 100000, 'val_size': 4000, 'test_size': 20000, 'is_bow': True, 'bow_strategy': 'indiv_topk', 'indiv_topk': 10, 'topk': 50, 'lr': '1e-3', 'only_bow': False, 'replace_predict': False, 'append_bow': True}


In [4]:
model_name = config_path.split('/')[-1][:-5]
output_all_path = '../data/preprocess_all_{}.npy'.format(model_name)


In [7]:
# just init one dataset / bert model
dataset = Dataset(
    "train", train_size+val_size+test_size, 0, 0, bow_strategy=bow_strategy, topk=topk, indiv_topk=indiv_topk,
    only_bow=only_bow, use_origin=only_bow, replace_predict=replace_predict
)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 11.91 GiB total capacity; 303.27 MiB already allocated; 22.00 MiB free; 328.00 MiB reserved in total by PyTorch)

In [7]:
sentence_tensors = []

In [8]:
for seq1_tensor, seq2_tensor in tqdm(data_loader):
    seq1_tensor = seq1_tensor.view(-1)
    seq2_tensor = seq2_tensor.view(-1)
    sentence_tensors.append([seq1_tensor, seq2_tensor])

100%|██████████| 124000/124000 [47:22<00:00, 43.62it/s] 


In [9]:
arr = np.array(sentence_tensors, dtype=object)
np.save(output_all_path, arr)

# load_arr = np.load(output_all_path, allow_pickle=True)

In [5]:
print(dataset[train_size+val_size+1])

NameError: name 'dataset' is not defined